<a href="https://colab.research.google.com/github/olegsh60/RAG/blob/main/%D0%9F%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B5_RAG_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_LLM_Qwen_2_5-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

#Ответы на вопросы с помощью LLM модели Qwen 7B + RAG

Устанавливаем общие библиотеки

In [ ]:
# Библиотеки Langchain для взаимодействия с LLM
!pip install -U langchain langchain-community langchain-huggingface -q
# Библиотеки для вектороной DB FAISS и загрузки PDF
!pip install faiss-cpu pypdf -q
# Следующие три строки - установка библиотеки unsloth для использования квантованных LLM.
# Для Colab процесс имеет специфику по версиям библиотек. Если запускать не в Colab, то просто:
# !pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo -q
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth -q

##Алгоритм создания RAG системы
1). Document Loading

In [ ]:
# Загружаем файл "Вектореый поиск.pdf" для RAG, предварительно его нужно загрузить в Colab в раздел в левой пенели "Files"
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('./Векторный поиск.pdf')
documents = loader.load()
print("Число страниц: ", len(documents))

2). Text Splitting — это процесс разбиения длинного текста на меньшие фрагменты (чаще всего — по абзацам, предложениям или символам), которые:
	•	легче обработать (в модели есть лимит токенов),
	•	проще индексировать в retrieval-системах,
	•	позволяют точнее связывать вопрос с контекстом.

In [ ]:
# Определите Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

split_documents = splitter.split_documents(documents)

print("Число чанков: ", len(split_documents))


3). Embedding это способ представления текста (или другого объекта) в виде числового вектора, который сохраняет смысл.

Простыми словами:

Это способ «перевести» текст в язык чисел, который понимает нейросеть.

Текст.    |Вектор (упрощённо).          |
----------|-----------------------------|
“кошка”.  |[0.12, -0.85, 0.33, …, 0.09  ]
“собака”. |[0.11, -0.83, 0.30, …, 0.12] |
“молекула”|[-0.45, 0.91, -0.12, …, 0.44]|

Вектора “кошка” и “собака” будут похожи, а “молекула” — далека.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

#  cuda - Use the GPU (if available)
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cuda"}
)

4). Storage. FAISS: Facebook AI Similarity Search

FAISS (читается «фэйс») — это библиотека от Facebook AI, предназначенная для поиска ближайших векторов (similarity search) в большом массиве данных.
Что делает FAISS?
	•	Быстро находит похожие элементы по вектору запроса.
	•	Поддерживает миллионы и миллиарды векторов.
	•	Работает на CPU и GPU.
	•	Используется в retrieval, поиске, рекомендательных системах, кластеризации.

In [ ]:
from langchain.vectorstores import FAISS

db_embed = FAISS.from_documents(split_documents, hf_embeddings_model)
db_embed.save_local("faiss_db")

5). Retrieval.  
### Что такое Retrieval?

Retrieval (англ. извлечение) — это процесс поиска и возврата релевантной информации из большой базы данных или набора документов в ответ на запрос пользователя.

В отличие от генерации (когда модель сама «придумывает» ответ), retrieval ориентирован на нахождение уже существующего контента — точного, проверенного и хранимого где-то вне модели.

Как работает Retrieval-пайплайн?

1. Индексирование
	•	Все документы конвертируются в векторы (embeddings).
	•	Эти векторы сохраняются в векторную базу данных (например, FAISS, Qdrant, Weaviate).

2. Запрос
	•	Ввод пользователя также превращается в вектор.

3. Поиск ближайших документов
	•	Система находит наиболее похожие вектора-документы (на основе косинусного расстояния или L2).

4. Подача в LLM
	•	Извлечённые документы добавляются в prompt.
	•	Модель генерирует обоснованный ответ.

In [ ]:
# Используем векторноое хранилище и его методов для получения документов
retriever = db_embed.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)


## Работа RAG системы


Импорт необходимых обьектов

In [ ]:
from unsloth import FastLanguageModel
from transformers import pipeline
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import StrOutputParser

Будем использовать Qwen 2.5 с 7B параметрами квантованную  в 4bit
https://huggingface.co/unsloth/Qwen2.5-7B-bnb-4bit


In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)


Pipeline для формирования ответа LLM

In [ ]:

terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(".")]

pipe = pipeline('text-generation',
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=150, # обьем возвращаемых токенов
                repetition_penalty=1.2, # штрав за повторы в ответе немного увеличим от дефортной 1
                temperature=0.7, # Креативность уменьшим от дефортной 1 чтобы получать более точные ответы
                eos_token_id=terminators)

HF_model = HuggingFacePipeline(pipeline=pipe)

Формируем шаблон для промпта

In [ ]:
template = """
Отвечай на вопрос только используя следующий контекст:
{context}
Если ответа нет в контексте, то ответь: я не знаю!
Question: {question}
"""

# Определение шаблона промпта
prompt_template = PromptTemplate.from_template(template)

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

Запускаем RAG, очищаем ответ

In [ ]:
query = "Что такое вектореый поиск?"
context = retriever.get_relevant_documents(query)
context = format_docs(context)
response = HF_model.invoke(prompt_template.format(question=query, context=context))

output_parser = StrOutputParser()
string_result = output_parser.parse(response)

print(string_result)

Пример вопроса на тему не из документа

In [ ]:
# Генерация ответа не по контексту
query = "Какое расстояние до луны?"

context = retriever.get_relevant_documents(query)
context = format_docs(context)
response = HF_model.invoke(prompt_template.format(question=query, context=context))

output_parser = StrOutputParser()
string_result = output_parser.parse(response)

print(string_result)

Пример запроса к чистой модели без использования RAG

In [ ]:
# Определение шаблона промпта
prompt_template_short = PromptTemplate.from_template("Вопрос: {question}")

# Генерация ответа
query = "Что такое векторный поиск?"
response = HF_model.invoke(prompt_template_short.format(question=query))

print(response)

Пример вопроса к чистой модели на тему не из документа

In [ ]:
# Определение шаблона промпта
prompt_template_short = PromptTemplate.from_template("Вопрос: {question}")

# Генерация ответа
query = "Какое расстяние до луны?"
response = HF_model.invoke(prompt_template_short.format(question=query))

print(response)